In [2]:
# Import Required Libraries and Dataset
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math

pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', 10) 
filepath = "../Datasets/Soil Health.csv"
df = pd.read_csv(filepath)
display(df)

,Location,Soil pH,Soil Moisture (%),Soil Organic Matter (%),Soil Nitrogen Content (mg/kg),Soil Phosphorus Content (mg/kg),Soil Potassium Content (mg/kg),Average Daily Temperature (⁰C),Total Rainfall (mm),Crop Yield (tons/ha)
0,West,5.87,38.00,3.19,2407.00,31.60,480.00,26.70,1200.00,8.72
1,Central,6.42,43.90,5.17,1702.00,78.10,293.00,23.40,822.00,7.60
2,East,7.53,44.30,3.93,1545.00,61.00,357.00,20.10,601.00,6.53
3,Central,5.71,26.20,4.78,1541.00,34.20,383.00,20.60,1200.00,6.53
4,Central,7.17,45.50,3.41,1371.00,57.20,125.00,27.20,954.00,6.92
...,...,...,...,...,...,...,...,...,...,...
95,East,6.58,20.00,3.98,1371.00,37.10,320.00,21.10,1147.00,5.80
96,Central,7.20,11.60,4.30,300.00,36.90,240.00,24.90,1108.00,3.62
97,South,6.61,22.10,2.79,1366.00,5.80,430.00,26.30,1009.00,5.52
98,South,6.08,31.50,1.84,1241.00,52.00,237.00,24.70,400.00,4.48


In [4]:
# Dataset Profilling
def column_summary(df):
    summary_data = []
    for col_name in df.columns:
        col_dtype = df[col_name].dtype
        num_of_nulls = df[col_name].isnull().sum()
        num_of_non_nulls = df[col_name].notnull().sum()
        num_of_distinct_values = df[col_name].nunique()
        
        if num_of_distinct_values <= 10:
            distinct_values_counts = df[col_name].value_counts().to_dict()
        else:
            top_10_values_counts = df[col_name].value_counts().head(10).to_dict()
            distinct_values_counts = {k: v for k, v in sorted(top_10_values_counts.items(), key=lambda item: item[1], reverse=True)}

        summary_data.append({
            'col_name': col_name,
            'col_dtype': col_dtype,
            'num_of_nulls': num_of_nulls,
            'num_of_non_nulls': num_of_non_nulls,
            'num_of_distinct_values': num_of_distinct_values,
            'distinct_values_counts': distinct_values_counts
        })
    
    summary_df = pd.DataFrame(summary_data)
    return summary_df
    
summary_df = column_summary(df)
display(summary_df)

,col_name,col_dtype,num_of_nulls,num_of_non_nulls,num_of_distinct_values,distinct_values_counts
0,Location,object,0,100,5,"{'West': 26, 'South': 21, 'Central': 19, 'Nort..."
1,Soil pH,float64,0,100,80,"{5.91: 3, 6.4: 3, 6.44: 2, 6.42: 2, 7.02: 2, 5..."
2,Soil Moisture (%),float64,0,100,84,"{36.8: 3, 43.9: 2, 45.5: 2, 33.2: 2, 22.2: 2, ..."
3,Soil Organic Matter (%),float64,0,100,94,"{4.34: 3, 3.35: 2, 3.45: 2, 3.18: 2, 4.79: 2, ..."
4,Soil Nitrogen Content (mg/kg),float64,0,100,94,"{1371.0: 2, 2295.0: 2, 1824.0: 2, 1062.0: 2, 1..."
5,Soil Phosphorus Content (mg/kg),float64,0,100,91,"{52.0: 3, 17.2: 2, 43.8: 2, 43.4: 2, 37.1: 2, ..."
6,Soil Potassium Content (mg/kg),float64,0,100,87,"{364.0: 3, 376.0: 3, 212.0: 3, 132.0: 2, 279.0..."
7,Average Daily Temperature (⁰C),float64,0,100,73,"{18.0: 4, 32.0: 4, 22.5: 3, 22.7: 3, 28.4: 2, ..."
8,Total Rainfall (mm),float64,0,100,77,"{400.0: 10, 1200.0: 4, 1139.0: 3, 601.0: 2, 87..."
9,Crop Yield (tons/ha),float64,0,100,92,"{5.65: 3, 6.53: 2, 5.29: 2, 5.78: 2, 7.67: 2, ..."


In [6]:
# Checking Missing Values
def Missig_values_info(df):   
    isna_df = df.isna().sum().reset_index(name='Missing Values Counts')
    isna_df['Proportions (%)'] = isna_df['Missing Values Counts']/len(df)*100
    return isna_df
    
isna_df = Missig_values_info(df)
isna_df

,index,Missing Values Counts,Proportions (%)
0,Location,0,0.00
1,Soil pH,0,0.00
2,Soil Moisture (%),0,0.00
3,Soil Organic Matter (%),0,0.00
4,Soil Nitrogen Content (mg/kg),0,0.00
5,Soil Phosphorus Content (mg/kg),0,0.00
6,Soil Potassium Content (mg/kg),0,0.00
7,Average Daily Temperature (⁰C),0,0.00
8,Total Rainfall (mm),0,0.00
9,Crop Yield (tons/ha),0,0.00


In [7]:
# Statistic Description of The Datasets
summary_stats = df.describe().T.reset_index()
summary_stats

,index,count,mean,std,min,25%,50%,75%,max
0,Soil pH,100.00,6.52,0.70,5.14,5.96,6.44,7.03,8.50
1,Soil Moisture (%),100.00,30.24,11.48,10.20,21.70,29.45,39.82,49.80
2,Soil Organic Matter (%),100.00,4.14,1.47,1.00,3.18,3.96,4.86,8.88
3,Soil Nitrogen Content (mg/kg),100.00,1441.93,488.10,300.00,1128.00,1427.50,1776.00,2456.00
4,Soil Phosphorus Content (mg/kg),100.00,42.21,16.75,5.80,32.27,43.25,52.00,78.60
5,Soil Potassium Content (mg/kg),100.00,320.54,118.46,50.00,247.50,336.00,382.25,617.00
6,Average Daily Temperature (⁰C),100.00,24.96,3.68,18.00,22.38,25.10,28.05,32.00
7,Total Rainfall (mm),100.00,823.53,235.85,400.00,664.50,829.00,1013.25,1200.00
8,Crop Yield (tons/ha),100.00,5.96,1.27,2.35,4.96,5.86,6.91,8.72


In [8]:
# Distribution of Continuous variables
def distribution_statistics(df, group_col=None):
    results = []
    if group_col: grouped = df.groupby(group_col)
    else: grouped = [(None, df)]

    for group_name, group_df in grouped:
        for col in group_df.select_dtypes(include=[np.number]).columns:
            mean = group_df[col].mean()
            median = group_df[col].median()
            mode = group_df[col].mode().iloc[0] if not group_df[col].mode().empty else np.nan
            std_dev = group_df[col].std()
            variance = group_df[col].var()
            value_range = group_df[col].max() - group_df[col].min()
            skewness_val = skew(group_df[col], nan_policy='omit')  # Skewness
            kurtosis_val = kurtosis(group_df[col], nan_policy='omit')  # Kurtosis
    
            results.append({
                'Group': group_name,
                'Parameter': col,
                'Mean': mean,
                'Median': median,
                'Mode': mode,
                'Standard Deviation': std_dev,
                'Variance': variance,
                'Range': value_range,
                'Skewness': skewness_val,
                'Kurtosis': kurtosis_val
            })
    
        
    result_df = pd.DataFrame(results)
    return result_df

pd.set_option('display.max_columns', 10) 
results = distribution_statistics(df)
display(results)

,Group,Parameter,Mean,Median,Mode,Standard Deviation,Variance,Range,Skewness,Kurtosis
0,None,Soil pH,6.52,6.44,5.91,0.70,0.50,3.36,0.41,-0.17
1,None,Soil Moisture (%),30.24,29.45,36.80,11.48,131.76,39.60,0.10,-1.10
2,None,Soil Organic Matter (%),4.14,3.96,4.34,1.47,2.17,7.88,0.49,0.24
3,None,Soil Nitrogen Content (mg/kg),1441.93,1427.50,300.00,488.10,238243.42,2156.00,-0.08,-0.39
4,None,Soil Phosphorus Content (mg/kg),42.21,43.25,52.00,16.75,280.46,72.80,0.05,-0.39
5,None,Soil Potassium Content (mg/kg),320.54,336.00,212.00,118.46,14032.61,567.00,-0.20,-0.11
6,None,Average Daily Temperature (⁰C),24.96,25.10,18.00,3.68,13.55,14.00,-0.03,-0.78
7,None,Total Rainfall (mm),823.53,829.00,400.00,235.85,55625.42,800.00,-0.21,-0.90
8,None,Crop Yield (tons/ha),5.96,5.86,5.65,1.27,1.61,6.37,-0.06,-0.39


In [9]:
# Group Wise Comparison of Distribution of Continuous variables
results = distribution_statistics(df, group_col='Location')
display(results)

,Group,Parameter,Mean,Median,Mode,Standard Deviation,Variance,Range,Skewness,Kurtosis
0,Central,Soil pH,6.53,6.42,6.42,0.54,0.29,2.17,0.74,0.21
1,Central,Soil Moisture (%),33.07,35.10,11.60,13.33,177.57,38.20,-0.23,-1.37
2,Central,Soil Organic Matter (%),4.21,4.24,1.70,1.35,1.82,4.82,-0.04,-0.74
3,Central,Soil Nitrogen Content (mg/kg),1564.26,1541.00,300.00,500.73,250734.54,2156.00,-0.40,0.56
4,Central,Soil Phosphorus Content (mg/kg),47.46,43.90,15.10,17.54,307.74,63.50,0.34,-0.53
5,Central,Soil Potassium Content (mg/kg),297.05,293.00,50.00,137.34,18862.61,450.00,-0.19,-0.88
6,Central,Average Daily Temperature (⁰C),24.37,24.90,18.20,3.48,12.11,12.30,-0.14,-0.87
7,Central,Total Rainfall (mm),818.21,826.00,400.00,285.92,81750.18,800.00,-0.26,-1.30
8,Central,Crop Yield (tons/ha),6.46,6.22,3.62,1.21,1.46,4.92,-0.13,0.05
9,East,Soil pH,6.63,6.78,7.02,0.65,0.43,2.21,-0.83,-0.47


In [10]:
# Group-wise Comparatives Analysis of Continuous variables
def summary_stats(df, group):
    Metrics = df.select_dtypes(include=np.number).columns.tolist()
    df_without_location = df.drop(columns=[group])
    grand_mean = df_without_location[Metrics].mean()
    sem = df_without_location[Metrics].sem()
    cv = df_without_location[Metrics].std() / df_without_location[Metrics].mean() * 100
    grouped = df.groupby(group)[Metrics].agg(['mean', 'sem']).reset_index()
    
    summary_df = pd.DataFrame()
    for col in Metrics:
        summary_df[col] = grouped.apply(
            lambda x: f"{x[(col, 'mean')]:.2f} ± {x[(col, 'sem')]:.2f}", axis=1
        )
    
    summary_df.insert(0, group, grouped[group])
    grand_mean_row = ['Grand Mean'] + grand_mean.tolist()
    sem_row = ['SEM'] + sem.tolist()
    cv_row = ['%CV'] + cv.tolist()
    
    summary_df.loc[len(summary_df)] = grand_mean_row
    summary_df.loc[len(summary_df)] = sem_row
    summary_df.loc[len(summary_df)] = cv_row
    
    return summary_df

results = summary_stats(df, group='Location')
results.T

,0,1,2,3,4,5,6,7
Location,Central,East,North,South,West,Grand Mean,SEM,%CV
Soil pH,6.53 ± 0.12,6.63 ± 0.16,6.80 ± 0.19,6.27 ± 0.18,6.44 ± 0.13,6.52,0.07,10.80
Soil Moisture (%),33.07 ± 3.06,34.64 ± 2.74,29.87 ± 2.97,26.50 ± 2.07,28.75 ± 2.05,30.24,1.15,37.96
Soil Organic Matter (%),4.21 ± 0.31,4.20 ± 0.37,4.19 ± 0.48,3.93 ± 0.31,4.19 ± 0.24,4.14,0.15,35.60
Soil Nitrogen Content (mg/kg),1564.26 ± 114.88,1248.12 ± 137.12,1409.78 ± 143.09,1398.95 ± 83.47,1528.77 ± 80.67,1441.93,48.81,33.85
Soil Phosphorus Content (mg/kg),47.46 ± 4.02,48.66 ± 4.09,43.18 ± 4.04,41.42 ± 3.56,34.38 ± 2.80,42.21,1.67,39.67
Soil Potassium Content (mg/kg),297.05 ± 31.51,323.25 ± 14.80,290.39 ± 29.34,367.76 ± 20.10,318.77 ± 27.46,320.54,11.85,36.96
Average Daily Temperature (⁰C),24.37 ± 0.80,24.96 ± 0.93,25.66 ± 0.87,26.01 ± 0.86,24.08 ± 0.70,24.96,0.37,14.74
Total Rainfall (mm),818.21 ± 65.59,784.00 ± 65.40,853.17 ± 56.61,801.57 ± 42.24,848.96 ± 43.16,823.53,23.59,28.64
Crop Yield (tons/ha),6.46 ± 0.28,5.68 ± 0.35,5.95 ± 0.31,5.56 ± 0.24,6.10 ± 0.25,5.96,0.13,21.27
